In [15]:
from collections import defaultdict
import pandas as pd
from surprise import Reader, Dataset
from surprise import KNNWithMeans, KNNBasic, SVD
from surprise import accuracy
from surprise.model_selection import train_test_split, cross_validate

import matplotlib.pyplot as plt
import numpy as np
import json

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
class RefinedMyAlgo():
    def __init__(self, rating_data='', data_frame='', movie_data=''):
        if rating_data:
            reader = Reader(line_format='user item rating timestamp', sep=',')
            self.ratings = Dataset.load_from_file(rating_data, reader)
#             self.trainset, self.testset = train_test_split(self.ratings, test_size=0.25)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
        elif not data_frame.empty:
            reader = Reader(rating_scale=(0, 5))
            self.ratings = Dataset.load_from_df(data_frame[['userId', 'movieId', 'rating']], reader)
            self.trainset = self.ratings.build_full_trainset()
            self.sim_options = {'name': 'cosine','user_based': False}
            
        if movie_data:
            self.movies = pd.read_csv(movie_data, low_memory=False)
            self.movies['year'] = self.movies['title'].apply(lambda x: x[-5:-1])
            self.movies['title'] = self.movies['title'].apply(lambda x: x[:-7])
            self.movies['genres'] = self.movies['genres'].apply(lambda x: x.replace('|',', '))

        
    def set_k(self, k_value=''):
        if k_value:
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            self.algo = algo
            self.algo.fit(self.trainset)
        else:
            algo = SVD()
            self.algo = algo
            self.algo.fit(self.trainset)
        
        
    def find_best_k(self, k_value=''):
        if k_value:
            print('K = {}'.format(k_value))
            algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
            return cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=True)
        else:
            aux = []
            for k_value in [3, 5, 7, 10, 15, 20, 30, 40]:
                print('K = {}'.format(k_value))
                algo = KNNWithMeans(k=k_value, sim_options=self.sim_options)
                my_dict = cross_validate(algo, self.ratings, measures=['RMSE', 'MAE'], cv=10, verbose=False)
                my_dict['k_value'] = k_value
                aux.append(my_dict)
            return aux
    
    
    def set_testset(self, users):
        if users:
            user_ratings = self.trainset.ur
            movies_ids = list(self.movies['movieId'])
            global_mean=self.trainset.global_mean
            my_testset = []
            
            for user in users:
                iuid = self.trainset.to_inner_uid(str(user))
                for movie in movies_ids:
                    is_in = False
                    for rating in user_ratings[iuid]:
#                         print( 'MOVIE: {}, RATING: {}'.format(movie,bla.trainset.to_raw_iid(rating[0])) )
                        if int(movie) == int(self.trainset.to_raw_iid(int(rating[0]))):
                            is_in = True
                            break
                    if not is_in:
                        my_tuple = (str(user),str(movie),global_mean)
                        my_testset.append(my_tuple)
                        
            self.testset = my_testset
        else:
            testset = self.trainset.build_anti_testset()
            self.testset = testset
        return self.testset


    def predict_ratings(self,users=''):
        # # Predict ratings for all pairs (u, i) that are NOT in the training set.
#         testset = self.trainset.build_anti_testset()
#         self.testset = testset
        testset = self.set_testset(users)
        predictions = self.algo.test(testset)
        self.predictions = predictions
        
        
    def set_perfil_movies(self, users):
        metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
        metadata = metadata.drop(columns="timestamp")

        metadata_filtered = metadata[metadata.userId.isin(users)]

        self.group_sparse_mtx = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
        
        self.perfil_movies = list(self.group_sparse_mtx)
        
    
    ### You must call self.set_perfil_movies() before
    def set_candidate_movies(self):
        candidate_movies = []
        for item in refinedMyAlgo.movies.iterrows():
        #     get the movieId of each movie in movies dataframe
            if item[1].values[0] not in self.perfil_movies:
                candidate_movies.append(item[1].values[0])
        self.candidate_movies = candidate_movies
        
        
    def calc_similarity_matrix(self):
        #Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
        tfidf = TfidfVectorizer(stop_words='english')
        
        #Replace NaN with an empty string
        self.movies['title'] = self.movies['title'].fillna('')
        self.movies['genres'] = self.movies['genres'].fillna('')
        
        #Construct the required TF-IDF matrix by fitting and transforming the data
        tfidf_matrix_title = tfidf.fit_transform(self.movies['title'])
        tfidf_matrix_genres = tfidf.fit_transform(self.movies['genres'])
        
        #Compute the cosine similarity matrix
        self.cosine_sim_movies_title = cosine_similarity(tfidf_matrix_title, tfidf_matrix_title)
        self.cosine_sim_movies_genres = cosine_similarity(tfidf_matrix_genres, tfidf_matrix_genres)
        
        
    def get_similar_movies(self, references, title_weight=0.5):
        recs = []
        for movie in references:
            # Get the pairwsie similarity scores of all movies with that movie
            movie_idx = int(self.movies[self.movies['movieId']==movie['movieID']].index[0])
            sim_scores_title = list(enumerate(self.cosine_sim_movies_title[movie_idx]))
            sim_scores_genres = list(enumerate(self.cosine_sim_movies_genres[movie_idx]))
            
            # Calculate total similarity based on title and genres
            total_sim_score = []
            for i in range(len(sim_scores_title)):
                aux = (sim_scores_title[i][1]*title_weight) + (sim_scores_genres[i][1]*(1-title_weight))
                total_sim_score.append((i, aux))
                
            # Sort the movies based on the similarity scores
            total_sim_score = sorted(total_sim_score, key=lambda x: x[1], reverse=True)
            
            candidates_sim_score = []
            for item in total_sim_score:
                if self.movies.loc[item[0]].values[0] not in self.perfil_movies:
                    candidates_sim_score.append(item)
            
            # Get the scores of the 10 most similar movies
            candidates_sim_score = candidates_sim_score[1:11]
            
            recs.append(candidates_sim_score)
            
        return recs
    
    
    def get_relevance_score(self, recs, references):
        count = 0
        recs_dict = []
        for reference in references:
        #     print('Referência: {}\t gêneros: {}'.format(refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][1], refinedMyAlgo.movies[refinedMyAlgo.movies['movieId']==reference['movieID']].values[0][2]))

            for movie in recs[count]:
                aux = {}

                movie_id = self.movies.loc[movie[0]].values[0]
                movie_title = self.movies.loc[movie[0]].values[1]
                movie_genres = self.movies.loc[movie[0]].values[2]
                movie_similarity = movie[1]
                movie_relevance = ( reference['rating']/5.0 )*movie_similarity

                aux['movie_id'] = movie_id
                aux['movie_title'] = movie_title
                aux['movie_genres'] = movie_genres
                aux['movie_similarity'] = movie_similarity
                aux['movie_relevance'] = movie_relevance

                recs_dict.append(aux)

        #         print('\tSim: {},\trelevance: {},\tmovieId: {},\ttitle: {}'.format(aux['movie_similarity'], aux['movie_relevance'], aux['movie_id'], aux['movie_title']))

            count=count+1

        recs_dict = sorted(recs_dict, key = lambda i: i['movie_relevance'],reverse=True)

        return recs_dict

In [17]:
refinedMyAlgo = RefinedMyAlgo(rating_data='ml-latest-small/ratings.csv', movie_data='ml-latest-small/movies.csv')
refinedMyAlgo.set_k()

In [18]:
my_users = [77,596,452,243,420]

refinedMyAlgo.predict_ratings(users=my_users)
len(refinedMyAlgo.predictions)

47891

In [19]:
# metadata = pd.read_csv('ml-latest-small/ratings.csv', low_memory=False, names=['userId', 'movieId', 'rating','timestamp'])
# metadata = metadata.drop(columns="timestamp")

# metadata_filtered = metadata[metadata.userId.isin(my_users)]

# my_group_sparse = pd.pivot_table(metadata_filtered, values='rating', index=['userId'], columns=['movieId'], fill_value=0)
# my_group_sparse.head()

refinedMyAlgo.set_perfil_movies(users=my_users)
refinedMyAlgo.set_candidate_movies()

# print(refinedMyAlgo.perfil_movies)
# print(refinedMyAlgo.candidate_movies)
refinedMyAlgo.group_sparse_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,0,0,0.0,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
243,0,5,0.0,0,4,0,4,0,4,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
420,4,0,3.5,0,0,0,0,0,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
452,0,4,0.0,0,0,0,4,5,0,0.0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0,0
596,4,0,3.5,4,0,4,0,0,0,3.5,...,2.5,3.5,3.5,4,3.5,3.5,3.5,3.5,4,4


In [20]:
group_filled_mtx = refinedMyAlgo.group_sparse_mtx.copy()

for index, row in group_filled_mtx.iterrows():
    for col in list(group_filled_mtx):
        if(group_filled_mtx.loc[index,col] == 0.0):
            aux = list(filter(lambda x: x.uid==str(index) and x.iid==str(col), refinedMyAlgo.predictions))
            group_filled_mtx.loc[index,col] = aux[0].est

group_filled_mtx.head()

movieId,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
userId,,,,,,,,,,,,,,,,,,,,,
77,4.394148,3.574105,4.303588,3.738375,3.494961,3.574300,2.741059,4.256997,3.367808,4.298014,...,3.723632,3.354142,3.666679,3.455356,3.463087,3.565072,3.263137,3.440476,3.636490,3.654128
243,4.420212,5.000000,4.536696,4.399350,4.000000,4.084055,4.000000,4.979501,4.000000,4.838844,...,4.061097,4.103315,4.132924,4.304123,4.132860,4.313426,4.309558,4.224044,4.303861,4.208050
420,4.000000,3.501481,3.500000,3.779248,3.763889,3.543199,2.729839,4.023256,3.336338,4.329507,...,3.561748,3.413677,3.513734,3.651637,3.292342,3.553483,3.609532,3.602912,3.711641,3.634068
452,4.819636,4.000000,4.745453,4.478584,4.785308,4.562498,4.000000,5.000000,4.315619,5.000000,...,4.308264,4.341893,4.327983,4.412334,4.111893,4.456376,4.447844,4.426000,4.436002,4.624554
596,4.000000,3.558542,3.500000,4.000000,3.744001,4.000000,2.688090,3.879571,3.194633,3.500000,...,2.500000,3.500000,3.500000,4.000000,3.500000,3.500000,3.500000,3.500000,4.000000,4.000000


In [21]:
########################################################################
# # Implementing least misery ending-up in a dataframe
########################################################################
values = []
labels = []
for i in range(0,len(list(group_filled_mtx))):
    my_col = group_filled_mtx.iloc[ : ,i]
    label = my_col.name
    my_col = list(my_col)
    
    labels.append(label)
    values.append( float(min(my_col)) )
    
# print('Array values: {}, Array labels: {}'.format(values, labels))
agg_group_perf = pd.DataFrame(index=[900], columns=labels)

for i in range(0,len(list(agg_group_perf))):
    agg_group_perf.iloc[0, i] = values[i]

agg_group_perf.head()

,1,10,32,34,36,39,44,47,48,50,...,176101,177763,178615,179401,179819,181719,182793,183635,184997,188301
900,4,3.50148,3.5,3.73837,3.49496,3.5432,2.68809,3.87957,3.19463,3.5,...,2.5,3.35414,3.5,3.45536,3.29234,3.5,3.26314,3.44048,3.63649,3.63407


In [22]:
group_pref_dict = []
for col in list(agg_group_perf):
    my_dict = {}
#     print('Valor: {}, Coluna: {}'.format(agg_group_perf.loc[900,col], col))
    my_dict['rating'] = agg_group_perf.loc[900,col]
    my_dict['movieID'] = col
    group_pref_dict.append(my_dict)
    
group_pref_dict = sorted(group_pref_dict, key = lambda i: i['rating'],reverse=True)
group_pref_dict

[{'rating': 4.2767357724406905, 'movieID': 318},
 {'rating': 4.152445762277182, 'movieID': 60069},
 {'rating': 4.145648363902219, 'movieID': 2160},
 {'rating': 4.1449668164140245, 'movieID': 904},
 {'rating': 4.1330889090840905, 'movieID': 168252},
 {'rating': 4.122718293860364, 'movieID': 1259},
 {'rating': 4.101869048838838, 'movieID': 6350},
 {'rating': 4.07653524959058, 'movieID': 44555},
 {'rating': 4.075970728185055, 'movieID': 608},
 {'rating': 4.0723876001272235, 'movieID': 1089},
 {'rating': 4.036221254267579, 'movieID': 4011},
 {'rating': 4.029943272122485, 'movieID': 910},
 {'rating': 4.02050592838333, 'movieID': 51255},
 {'rating': 4.013881679660402, 'movieID': 76093},
 {'rating': 4.003275548782965, 'movieID': 1617},
 {'rating': 4.002415707949423, 'movieID': 31658},
 {'rating': 4.0, 'movieID': 1},
 {'rating': 4.0, 'movieID': 260},
 {'rating': 4.0, 'movieID': 296},
 {'rating': 4.0, 'movieID': 593},
 {'rating': 4.0, 'movieID': 750},
 {'rating': 4.0, 'movieID': 1028},
 {'ratin

In [23]:
refinedMyAlgo.calc_similarity_matrix()

In [24]:
references = group_pref_dict[0:10]
# references = group_pref_dict
print(references)

[{'rating': 4.2767357724406905, 'movieID': 318}, {'rating': 4.152445762277182, 'movieID': 60069}, {'rating': 4.145648363902219, 'movieID': 2160}, {'rating': 4.1449668164140245, 'movieID': 904}, {'rating': 4.1330889090840905, 'movieID': 168252}, {'rating': 4.122718293860364, 'movieID': 1259}, {'rating': 4.101869048838838, 'movieID': 6350}, {'rating': 4.07653524959058, 'movieID': 44555}, {'rating': 4.075970728185055, 'movieID': 608}, {'rating': 4.0723876001272235, 'movieID': 1089}]


In [25]:
# references=[904, 318, 38061, 31658, 57669]
recs = refinedMyAlgo.get_similar_movies(references)

In [26]:
final_list = refinedMyAlgo.get_relevance_score(recs=recs, references=references)

for item in final_list[0:20]:
    print('Recs_dict: relevance: {}, title:{}'.format(item['movie_relevance'], item['movie_title']))

Recs_dict: relevance: 0.5449157824743178, title:Baby, The
Recs_dict: relevance: 0.4823436813951736, title:The Raid: Redemption
Recs_dict: relevance: 0.42767357724406907, title:Casino
Recs_dict: relevance: 0.42767357724406907, title:Shanghai Triad (Yao a yao yao dao waipo qiao)
Recs_dict: relevance: 0.42767357724406907, title:Hate (Haine, La)
Recs_dict: relevance: 0.42767357724406907, title:Young Poisoner's Handbook, The
Recs_dict: relevance: 0.42767357724406907, title:Heavenly Creatures
Recs_dict: relevance: 0.42767357724406907, title:New Jersey Drive
Recs_dict: relevance: 0.42767357724406907, title:Once Were Warriors
Recs_dict: relevance: 0.42767357724406907, title:Jason's Lyric
Recs_dict: relevance: 0.42767357724406907, title:Above the Rim
Recs_dict: relevance: 0.41979456149008365, title:Stand and Deliver
Recs_dict: relevance: 0.4145648363902219, title:Mary Reilly
Recs_dict: relevance: 0.4145648363902219, title:Innocents, The
Recs_dict: relevance: 0.4145648363902219, title:Body Snatc